In [247]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk

In [248]:
df_train = pd.read_csv('balanced_2classes_train.csv', encoding = 'utf-8')
df_test = pd.read_csv('balanced_2classes_test.csv', encoding = 'utf-8')
df_val = pd.read_csv('balanced_2classes_val.csv', encoding = 'utf-8')

In [249]:
stops = pd.read_excel('ar_stops.xlsx', encoding = 'utf-8')
type(stops)
type(stops[0:])

pandas.core.frame.DataFrame

In [250]:
stops.columns

Index(['“'], dtype='object')

In [251]:
stop_words = list(stops['“'])

In [252]:
df_train.head()

,label,user_id,book_id,text
0,0,4969402,10706934,لا جديد بالنسبة لي!
1,0,4865181,3153116,دعونا لا ننكر أن لغة علوان هنا ، جمـيلة جداً ...
2,0,8132581,3831344,الكتاب كان دون توقعاتي لعله بسبب كرهي للفلسفة...
3,0,8553746,11205152,مشدنيش في حاجة . وسلبي بشكل كبير ويميل للكآبه...
4,0,5115054,11942408,شغاال برضه بس مش واو!!


In [253]:
total = pd.concat([df_train, df_test, df_val], axis=0)

In [254]:
total.shape

(16448, 4)

In [255]:
total.head()

,label,user_id,book_id,text
0,0,4969402,10706934,لا جديد بالنسبة لي!
1,0,4865181,3153116,دعونا لا ننكر أن لغة علوان هنا ، جمـيلة جداً ...
2,0,8132581,3831344,الكتاب كان دون توقعاتي لعله بسبب كرهي للفلسفة...
3,0,8553746,11205152,مشدنيش في حاجة . وسلبي بشكل كبير ويميل للكآبه...
4,0,5115054,11942408,شغاال برضه بس مش واو!!


In [256]:
text = total['text']

In [257]:
!pip install PyArabic

In [258]:
import pyarabic.araby as araby
text = text.apply(araby.strip_tashkeel)
text = text.apply(araby.strip_harakat)
text = text.apply(araby.strip_tatweel)
text = text.apply(araby.normalize_hamza)

In [259]:
import re
def rem_punc(x):
    x = re.sub("\d+", "", x)
    return x

In [260]:
text = text.apply(rem_punc)

In [261]:
nltk.download('punkt')
from nltk.tokenize import word_tokenize
text = text.apply(lambda x: word_tokenize(x))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\CompuTek\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [262]:
text = text.apply(lambda words: [w for w in words if w not in stop_words] )

In [263]:
text.head()

0                                  [لا, جديد, بالنسبة]
1    [دعونا, لا, ننكر, ءن, لغة, علوان, جميلة, جدا, ...
2    [الكتاب, توقعاتي, كرهي, للفلسفة, ولعله, ءسلوب,...
3    [مشدنيش, حاجة, وسلبي, كبير, ويميل, للكءءبه, اكبر]
4                                [شغاال, برضه, بس, مش]
Name: text, dtype: object

In [264]:
text.shape

(16448,)

In [265]:
from nltk.stem.isri import ISRIStemmer
nltk.download('wordnet')
text = text.apply(lambda x: [ISRIStemmer().stem(w) for w in x])

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\CompuTek\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [266]:
def replace(x):
    re.sub("n't","","not".join(x))
    return x 
            

In [267]:
text = text.apply(replace)

In [268]:
text = text.apply(lambda x: " ".join(x))

In [269]:
text.head()

0                                           لا جدد نسب
1    دعو لا نكر ءن لغة علو جمل جدا لا خفى علي ءني ب...
2    كتب وقع كره لسف لعل ءسلوب د.المسيري تقعر عالءق...
3                      مشدنيش حجة سلب كبر يمل كءءب كبر
4                                       شغال برض بس مش
Name: text, dtype: object

In [270]:
train = total[:df_train.shape[0]]
test = total[df_train.shape[0]:df_train.shape[0] + df_test.shape[0]]
val = total[df_train.shape[0] +df_test.shape[0] :]

In [271]:
train.shape[0] + val.shape[0] + test.shape[0]

16448

In [293]:
## Using TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf = TfidfVectorizer(ngram_range=(1, 2), min_df=2, max_df=.5)
X_train = tf_idf.fit_transform(train['text'])

In [294]:
X_val = tf_idf.transform(val['text'])
X_test = tf_idf.transform(test['text'])

In [295]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, train['label'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [296]:
clf.score(X_train, train['label'])

0.9549230145867099

In [297]:
predict_val = clf.predict(X_val)

In [298]:
from sklearn.metrics import accuracy_score
accuracy_score(val['label'], predict_val)

0.8038321167883211

In [299]:
predict_test = clf.predict(X_test)
accuracy_score(test['label'], predict_test)

0.8159975669099757